In [40]:
import pandas as pd


In [2]:
filename = '/home/hb/machinelearning/Realtime-Fall-Detection-for-RNN/dataset/MobiFall_Dataset_v2.0/sub1/FALLS/SDL/SDL_ori_1_3.txt'

In [3]:
df =  pd.read_csv(filename, skiprows=16, header=None)

In [4]:
df.head()

,0,1,2,3
0,2263734564000,231.59665,8.406437,-65.78639
1,2263739503000,231.88106,10.277120,-68.14765
2,2263744482000,231.48561,8.862655,-65.99789
3,2263749482000,258.91132,11.825000,-66.32058
4,2263754447000,260.28268,12.254481,-66.41949


By observing the dataset, we got 3 type of files for activity
['ori', 'acc', 'gyro']

filename = <b>'{actvity_name}__{feature_type}_{filename}.txt'<b>

For Example=> BSC_gyro_1_1.txt


In [6]:
acc_columns = ['timestamp(ns)','x','y','z(m/s^2)']

In [7]:
ori_columns = ['timestamp(ns)', 'Azimuth' , 'Pitch', 'Roll']

In [8]:
gyro_columns = ['timestamp(ns)','x','y','z(rad/s)']

In [9]:
dataset_path = './dataset/MobiFall_Dataset_v2.0'


In [28]:
import glob
class DataGenerator():
    
    _data = []
    
    def __init__(self, dataset_path):
        self._datafiles = glob.glob('./dataset/*/*/*/*/*.txt')
        
        
    def get_data_files(self):
        return self._datafiles
    
    
        

In [29]:
generator = DataGenerator('mayur')

In [32]:
len(generator.get_data_files())

1890

In [36]:
generator.get_data_files()[0].split('/')[-1].split('_')

['BSC', 'ori', '15', '3.txt']

In [37]:
ADL, SENSOR_CODE, SUBJECT_ID, TRIAL_NO = generator.get_data_files()[0].split('/')[-1].split('_')

In [38]:
SUBJECT_ID

'15'

In [82]:
ori_Df = pd.DataFrame(columns=ori_columns)
acc_Df = pd.DataFrame(columns=acc_columns)
gyro_Df = pd.DataFrame(columns=gyro_columns)

for file in generator.get_data_files():
    ADL, SENSOR_CODE, SUBJECT_ID, TRIAL_NO = file.split('/')[-1].split('_')
    columns = None
    if SENSOR_CODE == 'acc':
        df = pd.read_csv(file, skiprows=16, names=acc_columns)
        df['activity'] = ADL
        df['subject_id'] = SUBJECT_ID
        acc_Df = acc_Df.append(df, ignore_index=True, sort=True)        
    elif SENSOR_CODE == 'ori':
        df = pd.read_csv(file, skiprows=16, names=ori_columns)
        df['activity'] = ADL
        df['subject_id'] = SUBJECT_ID
        ori_Df = ori_Df.append(df, ignore_index=True, sort=True)
    elif SENSOR_CODE == 'gyro':
        df = pd.read_csv(file, skiprows=16, names=gyro_columns)
        df['activity'] = ADL
        df['subject_id'] = SUBJECT_ID
        gyro_Df = gyro_Df.append(df, ignore_index=True, sort=True)
    else:
        print('Unwanted data detected!! => {}'.format(file))
    
    
        

In [83]:
ori_Df.head()

,Azimuth,Pitch,Roll,activity,subject_id,timestamp(ns)
0,19.273499,174.40622,31.021011,BSC,15,2571632215000
1,19.274391,174.41716,31.022657,BSC,15,2571638275000
2,19.495820,174.16460,31.328773,BSC,15,2571642239000
3,19.738337,173.89550,31.626282,BSC,15,2571647220000
4,19.999420,173.62091,31.921803,BSC,15,2571652295000


In [84]:
gyro_Df.head()

,activity,subject_id,timestamp(ns),x,y,z(rad/s)
0,BSC,15,2541169805000,-1.132239,0.722959,0.040928
1,BSC,15,2541174716000,-1.132239,0.744950,0.037263
2,BSC,15,2541179712000,-1.179581,0.737620,0.030849
3,BSC,15,2541184711000,-1.172250,0.735787,0.032376
4,BSC,15,2541189705000,-1.171334,0.743423,0.040623


In [85]:
ori_Df.head()

,Azimuth,Pitch,Roll,activity,subject_id,timestamp(ns)
0,19.273499,174.40622,31.021011,BSC,15,2571632215000
1,19.274391,174.41716,31.022657,BSC,15,2571638275000
2,19.495820,174.16460,31.328773,BSC,15,2571642239000
3,19.738337,173.89550,31.626282,BSC,15,2571647220000
4,19.999420,173.62091,31.921803,BSC,15,2571652295000


In [86]:
"Shape ori Df = "+str(ori_Df.shape)+ " Shape of acc Df = "+str(acc_Df.shape)+ " Shape of gyro Df "+str(gyro_Df.shape)

'Shape ori Df = (2370472, 6) Shape of acc Df = (1044765, 6) Shape of gyro Df (2389589, 6)'

In [87]:
ori_Df.describe()

,Azimuth,Pitch,Roll
count,2.370472e+06,2.370472e+06,2.370472e+06
mean,1.843732e+02,-6.973394e+01,-1.255405e+00
std,1.140415e+02,4.674030e+01,2.224753e+01
min,-8.285666e+01,-1.799994e+02,-8.621191e+01
25%,8.123130e+01,-9.686837e+01,-1.087106e+01
50%,1.865824e+02,-7.937894e+01,-4.656869e-01
75%,2.965372e+02,-5.688146e+01,8.308768e+00
max,3.600000e+02,1.799986e+02,8.982761e+01


In [88]:
gyro_Df.describe()

,x,y,z(rad/s)
count,2.389589e+06,2.389589e+06,2.389589e+06
mean,-2.410321e-02,-4.059009e-03,-8.268760e-03
std,1.021096e+00,1.002171e+00,6.803024e-01
min,-9.993755e+00,-1.000781e+01,-1.000597e+01
25%,-1.203405e-01,-1.444696e-01,-6.933320e-02
50%,3.054326e-04,3.665192e-03,1.038471e-02
75%,1.643227e-01,1.597413e-01,9.651671e-02
max,9.995282e+00,1.000811e+01,1.000689e+01


In [89]:
acc_Df.describe()

,x,y,z(m/s^2)
count,1.044765e+06,1.044765e+06,1.044765e+06
mean,-2.455672e-01,7.217378e+00,1.379892e+00
std,4.060810e+00,5.218344e+00,4.219554e+00
min,-1.950796e+01,-1.945050e+01,-1.949838e+01
25%,-2.231396e+00,3.035848e+00,-7.757214e-01
50%,-1.819593e-01,9.021352e+00,4.501099e-01
75%,1.589750e+00,9.883265e+00,3.763685e+00
max,1.953669e+01,1.959415e+01,1.953669e+01


In [90]:
acc_Df['subject_id'].unique()

array(['15', '7', '11', '2', '16', '21', '10', '29', '13', '8', '3', '12',
       '6', '20', '18', '9', '5', '14', '17', '1', '19', '30', '4', '31'],
      dtype=object)

In [91]:
dataDf = acc_Df.merge(gyro_Df, on='timestamp(ns)').merge(ori_Df, on='timestamp(ns)')

In [92]:
dataDf.shape

(320, 16)

In [93]:
dataDf.head()

,activity_x,subject_id_x,timestamp(ns),x_x,y_x,z(m/s^2),activity_y,subject_id_y,x_y,y_y,z(rad/s),Azimuth,Pitch,Roll,activity,subject_id
0,BSC,15,2580049700000,4.845864,-1.503559,-8.580819,STD,7,0.007330,0.018326,0.015272,270.563320,-93.043950,-0.623409,STD,7
1,BSC,15,894311041000,4.692636,2.030283,8.149862,SCH,5,0.014355,0.011301,0.020464,333.170600,-24.819588,-31.839170,SCH,5
2,FKL,15,2240665667000,-0.641646,-9.519346,0.488417,JOG,2,1.325578,-2.294715,0.999375,85.400700,-76.430850,2.340226,JOG,2
3,JOG,7,3414694749000,5.650316,7.134721,0.430956,FOL,16,-0.019242,0.013134,0.015882,205.229450,-166.525240,-30.087284,FOL,16
4,JOG,7,3318809102000,-0.536301,9.969456,5.008670,JOG,7,2.177124,0.127060,0.255647,67.742615,-103.563520,11.364722,JOG,7
